In [1]:
from scanr_utils import *
from casuhal_utils import *
import pandas as pd, json

In [2]:
# ____1____ RECUPERER LES DOI

## utiliser l'API de ScanR

id_etab = "0755976N" # identifiant UAI de votre établissement (le récupérer sur scanR par ex.)
min_year = 2021
# scanR inclut des publications jusqu'a 2020. min_year est l'année à partir de laquelle commencer la récupération
# renseigner 2021 revient à ne pas utiliser scanR. 
# choix effectué à UP pour favoriser dans un premier temps les DOI récents (extrait de scopus)
df_scanr = get_publications_with_doi(id_etab, min_year)
print(f"nb de DOI récupérés de ScanR \t{len(df_scanr)}")
#df_scanr.to_csv("data\doi_scanr.csv", columns= ["doi"], index = False)

nb de DOI récupérés de ScanR 	0


In [3]:
# option : ajouter des doi en plus de scanr ? 
ajout_doi_en_plus_de_scanr = True # True pour oui False pour non

if ajout_doi_en_plus_de_scanr : 
    # fichier csv doit être dans le dossier *data*
    # ',' comme séparateur, utf8 pour l'encodage, avec une colonne *doi*
    df_mydois = pd.read_csv("data\scopus_2021.csv", usecols=["doi"]) 
    print(f"nb de DOI importés depuis fichier \t{len(df_mydois)}")
    
    # concatener et dédoublonner
    df = pd.concat([df_scanr, df_mydois])
    df["doi"] = df["doi"].str.lower() # doi en minuscule
    df = df[ (~df['doi'].duplicated()) & (df["doi"].notna())].copy()
    df.reset_index(drop=True, inplace=True)

nb de DOI importés depuis fichier 	1283


In [4]:
if not ajout_doi_en_plus_de_scanr : 
    df = df_scanr
print(f"nb de DOI a traiter \t{len(df)}")

nb de DOI a traiter 	1282


In [5]:
# ____2____ ENRICHIR

## avec HAL
df_hal = enrich_w_hal(df) # renseigner df[:50].copy() pour tester uniquement sur les 50 premiers DOI
df_no_file = df_hal[ df_hal["submitType"] != "file" ].copy()
print(f"nb de DOI après retrait de ceux en TI Dans HAL {len(df_no_file)}")

nb DOI a verifier dans HAL 1282
hal 10% 
hal 20% 
hal 30% 
hal 40% 
hal 50% 
hal 60% 
hal 70% 
hal 80% 
hal 90% 
hal 100% 
hal 100%
nb de DOI après retrait de ceux en TI Dans HAL 1113


In [6]:
## avec unpaywall
df_upw = enrich_w_upw(df_no_file)

nb DOI a verifier dans upw 	1113
upw 10% 
upw 20% 
upw 30% 
upw 40% 
upw 50% 
upw 60% 
upw 70% 
upw 80% 
upw 90% 
upw 100% 
upw 100%


In [7]:
## avec Permissions
df_permissions = enrich_w_permissions(df_upw)

permissions 100%


In [8]:
## premier export des données brutes
df_permissions.to_csv("data\dois_hal_upw_perm.csv", index = False)

In [9]:
# ____3____ TRIE & FORMATAGE

df_final = df_permissions
df_final.fillna("", inplace=True)# sinon nan sera compris comme une string non vide

## retirer ce qui est dans HAL qui a un lien extérieur et qui est signalé en repository dans upw
index2remove = df_final[ (df_final["linkExtId"] != "") & (df_final["oa_repo_link"] != "") ].index
df_final.drop(index2remove, inplace = True)
print(f"nb DOI retiré car marqué 'open' dans HAL et repository dans upw {len(index2remove)}" )

nb DOI retiré car marqué 'open' dans HAL et repository dans upw 50


In [10]:
#deduire la colonne todo
df_final["todo"] = df_final.apply(lambda row : deduce_todo(row), axis = 1)


In [11]:
#retrait des colonnes inutiles 
df_final.drop(columns= ["has_issn", "submitType"], inplace = True)
df_final.to_csv("data\dois_a_traiter.csv", index = False)
#produire les statistiques brutes
print(f"\n\nStatistiques\n\nnb de DOI a traiter {len(df_final)}\n{df_final['todo'].value_counts()}")



Statistiques

nb de DOI a traiter 1063
mailto auteur pour appliquer LRN                        568
creer/retrouver notice                                  266
selon licence ajouter PDF editeur                       208
verifier identifiants notice                              5
recuperer pdf publisher et mailto auteur pour accord      5
Name: todo, dtype: int64


In [12]:
# rendre la lecture dans libreOffice plus élégante avec l'ajout d'hyperliens
df_final["doi"] = df_final.apply(lambda row : addCaclLinkFormula("https://doi.org/", row["doi"], row["doi"]), axis = 1)  
df_final["halId"] = df_final.apply(lambda row : addCaclLinkFormula("https://hal.archives-ouvertes.fr/", row["halId"], row["halId"]), axis = 1)  
df_final["oa_publisher_link"] = df_final.apply(lambda row : addCaclLinkFormula("", row["oa_publisher_link"], row["oa_publisher_link"]), axis = 1)	
df_final["oa_repo_link"] = df_final.apply(lambda row : addCaclLinkFormula("", row["oa_repo_link"], row["oa_repo_link"]), axis = 1)	

df_final.to_csv("data\dois_a_traiter_formula.csv", index = False)